# LinkedIn ETL

## Import Dependencies

In [1]:
from bs4 import BeautifulSoup as soup 
import pandas as pd
import pymongo 
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pantab

In [2]:
# Initialize Mongo 
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.thousand_tabs
collection = db.job_detail
job_links_collection = db.job_links

## Loading Data

In [3]:
# Getting the URL links into a DataFrame
url_links = pd.DataFrame(list(job_links_collection.find()))

In [4]:
# Load data from our job detail collection in our raw_html_db database
data = pd.DataFrame(list(collection.find()))

In [5]:
# Load Skill key words to compare to job descriptions
skill_net = pd.read_csv('../data/skill_list.csv')
skill_net_unique = pd.DataFrame(list(dict.fromkeys([x.strip().lower() for x in skill_net['skill_name']])), columns=['skill_net'])

In [6]:
# Create a list of Beautiful Soup objects to iterate
link_list = []
for html in data.link_src:
    link_list.append(soup(html,'html.parser'))

## Extracting Data

In [7]:
# Iterate through each sublist of html, parse it, save the result in list. 
# then later convert to a dataframe
headers = []
job_titles = []
senority = []
posted = []
employment_type = []
job_function = []
industry = []
job_summary = []
job_skills = []

# Loop through HTMLs in URL link list from MongoDB
for idx in range(len(link_list)):  
    
    # Extract Company Name and Company Location
    for x in link_list[idx].find_all('h3','jobs-top-card__company-info t-14 mt1'):
        vitals = x.get_text().replace('Company Name','').replace('Company Location','').split('\n')
        vitals = tuple([elem for elem in vitals if elem.strip()])
        headers.append({'company':vitals[0].strip(),'location':vitals[1].strip()})
    
    # Extract Job Titles
    try:
        for title in link_list[idx].find('h1',"jobs-top-card__job-title t-24"):
            job_titles.append(title)
    except:
        job_titles.append(None)
    
    # Extract Job Seniority (i.e. Junior, Senior, Director,etc.)
    try:
        for x in link_list[idx].find('p','jobs-box__body js-formatted-exp-body'):
            senority.append(x)
    except:
        senority.append(None)
    
    # Extract Job Posting Time (i.e. 1 Week Ago)
    try:
        for x in link_list[idx].find_all('p','mt1 full-width flex-grow-1 t-14 t-black--light'):
            posted.append(x.find_all('span')[1].text)
    except:
        posted.append(None)

    # Extract Employment Type (i.e. Full-Time, Contract, Internship, etc.)
    try:
        for x in link_list[idx].find('p','jobs-box__body js-formatted-employment-status-body'):
            employment_type.append(x)
    except:
        employment_type.append(None)

    # Extract Job Functions into seperate DataFrame (i.e. Accounting, IT, Finance, etc.)
    try:
        for x in link_list[idx].find_all('ul','jobs-box__list jobs-description-details__list js-formatted-job-functions-list'):
            for y in (x.get_text().strip('').split('\n')[1:-1]):
                job_function.append(((idx, y)))
    except:
        print(f'Job Function failed at {idx}')
    
    # Extract the Industry into seperate DataFrame (i.e. Entertainment, Media Production) 
    try:
        for x in link_list[idx].find_all('ul','jobs-box__list jobs-description-details__list js-formatted-industries-list'):
            for y in (x.get_text().strip('').split('\n')[1:-1]):
                industry.append(((idx, y)))
    except:
         print(f'Job Industry failed at {idx}')
            
###JOB SKILLS###

# Extract skills from job descriptions
# Making all words lowercase for easy comparison with our skill_net draglist
for idx in range(len(link_list)):
    for txt in link_list[idx].find_all('article','jobs-description__container jobs-description__container--condensed'):
        job_summary.append(txt.get_text().lower().replace('\n', ' ').strip())


# Add spaces behind and in front of words to find the exact word ('r' => ' r ')
skill_net_unique.skill_net =  pd.DataFrame(skill_net_unique.skill_net.map(lambda x: f' {x} '))    

# Filtering text data by job on words inside of our skills list
for idx, job in enumerate(job_summary):
    for skill in skill_net_unique['skill_net']:
        if skill in job:
            job_skills.append((idx, skill))

# Create job table
df1 = pd.DataFrame(headers)
df2 = pd.DataFrame(job_titles,columns=['job_title'])
df3 = pd.DataFrame(senority,columns=['experience_level'])
df4 = pd.DataFrame(posted,columns=['posted_time'])
df5 = pd.DataFrame(employment_type,columns=['employment_type'])
job_df = pd.concat([df2,df1,df3,df4,df5],axis=1)
job_df['scrape_time'] = datetime.datetime.now()
job_df = job_df.reset_index().rename(columns={'index':'job_id'})

# Create job function, industry and skills tables
job_function_df = pd.DataFrame(job_function, columns=['job_id', 'job_function'])
industry_df = pd.DataFrame(industry, columns=['job_id','industry'])
skills_df = pd.DataFrame(job_skills, columns=['job_id', 'skill'])

# Extract Linkedin job link url
# Create job url link table
links_df = url_links.reset_index().rename(columns={'index':'job_id'})
links_df.drop(columns='_id',inplace=True)
links_df['link'] = links_df['link'].map(lambda x: 'www.linkedin.com' + x)

In [18]:
df_dict = {'job': job_df,
           'job_function': job_function_df,
           'industry': industry_df,
           'skill': skills_df,
           'url': links_df}

In [19]:
def hyper_extract(df_dict):
    for key in df_dict:
        print(key)
        pantab.frame_to_hyper(df_dict[key], f"../data/{key}.hyper", table = f'{key}')
hyper_extract(df_dict)

job
job_function
industry
skill
url


# Plot Skills

In [ ]:
import pantab
pantab.frame_to_hyper(job_skills_df, "../data/job_skills.hyper", table = 'job_skill data')

In [ ]:
dataset['search_query'] = 'data scientist'
dataset['search_location'] = 'greater los angeles area'
test2 = dataset.drop_duplicates(subset=['job_title','company', 'location', 'posted_time'])
test2

In [ ]:
# tmp = []
# for job in range(len(link_list)):
#     try:
#         for detail in link_list[job].find_all('span','jobs-ppc-criteria__value'):
#             tmp.append((job,detail.get_text()))
#     except: 
#         print('no data here')
#         pass
# tmp

# final_job_skills_index  = pd.DataFrame(tmp,columns=['job_id','skills'])
# final_job_skills_index.skills = final_job_skills_index.skills.map(lambda x: x.strip())
# final_job_skills_index

In [ ]:
links_df = url_links.reset_index().rename(columns={'index':'job_id'})
links_df.drop(columns='_id',inplace=True)
links_df['link'] = links_df['link'].map(lambda x: 'www.linkedin.com' + x)
links_df

In [ ]:
# tester.drop(columns='_id',inplace=True)

In [ ]:
# tester['link'] = tester['link'].map(lambda x: 'www.linkedin.com' + x)
# tester = tester.merge(final_job_skills_index,on='job_id',how='left')
# tester[tester.skills.isnull()]
# final_job_skills_index.to_csv('../data/skills_indexed.csv')


In [ ]:
# tester

In [ ]:
# tester[tester.skills.isnull()]

In [ ]:

# link_list[1].find_all('li','jobs-box__list-item jobs-description-details__list-item')

In [ ]:
# from sqlalchemy import create_engine
# connection_string = 'postgres:postgres@localhost:5432/li_analysis'
# engine = create_engine(f'postgresql://{connection_string}')

In [ ]:
# engine.table_names()

In [ ]:
# df7.to_sql(name='industry', con=engine, if_exists='replace', index=False)
# df6.to_sql(name='job_function', con=engine, if_exists='replace', index=False)
# final_job_skills_index.to_sql(name='skill', con=engine, if_exists='replace', index=False)
# test.to_sql(name='job', con=engine, if_exists='replace', index=False)
# df7
# test2

In [ ]:
# test = dataset.reset_index().rename(columns={"index": "job_id"})

In [ ]:
# test2_df = pd.merge(test2.reset_index().rename(columns={"index": "job_id"}), final_job_skills_index, on=['job_id'])

In [ ]:
# test2_df.to_csv('../data/thousand_dataset.csv')

In [ ]:
# all_skills = pd.read_csv('../data/all_skills.csv')

In [ ]:
# master_merge = test.merge(all_skills,on='job_id')

In [ ]:
# master_merge.to_csv('../data/master_merged.csv')

In [ ]:
# test.to_csv('../data/jobs_table.csv')

In [ ]:
# master_merge.drop(columns='Unnamed: 0',inplace=True)

In [ ]:
# master_merge

In [ ]:
# lnks.drop(columns='_id',inplace=True)

In [ ]:
# lnks['job_id'] = np.arange(len(lnks))

In [ ]:
# master_merge_links = master_merge.merge(lnks, on='job_id')

In [ ]:
# master_merge|

In [ ]:
# master_merge_links

In [ ]:
# master_merge_links.link = ['www.linkedin.com' + x for x in master_merge_links['link']]

In [ ]:
# master_merge_links.to_csv('../data/master_merge_links.csv')

In [36]:
url_links['link'].iloc[667]

'/jobs/view/1797065172/?eBP=NotAvailableFromVoyagerAPI&recommendedFlavor=COMPANY_RECRUIT&refId=3f9ba05b-79e5-4ac9-90a7-d29855349eee&trk=d_flagship3_search_srp_jobs'

In [37]:
job_df.iloc[667]

job_id                                         667
job_title           Applied Scientist II - AMZ3410
company                                      Amgen
location                     Thousand Oaks, CA, US
experience_level                  Mid-Senior level
posted_time                     Posted 1 month ago
employment_type                          Full-time
scrape_time             2020-04-14 01:26:54.890756
Name: 667, dtype: object

In [83]:
tst = []
for idx in range(len(link_list)):
    try:
        for x in link_list[idx].find_all('meta'):
            print(x['href'])
    except:
        break

In [79]:
tst

[]

In [89]:
link_list[idx].find('a','jobs-top-card__company-url ember-view',href=True).get_text()

'          Focus GTS\n'